In [1]:
import pandas as pd


In [7]:
Recipe_temp = pd.DataFrame( {"Theme" : ["AC","DLPFC","PCC"] ,
                "genotype_list" : ["/home/hs3163/GIT/ADSPFG-xQTL/MWE/mwe_genotype_list",
                   "/home/hs3163/GIT/ADSPFG-xQTL/MWE/mwe_genotype_list",
                   "/home/hs3163/GIT/ADSPFG-xQTL/MWE/mwe_genotype_list"],
                "molecular_pheno" : ["/home/hs3163/Project/Rosmap/data/gene_exp/AC/geneTpmResidualsAgeGenderAdj_rename.txt",
                     "/home/hs3163/Project/Rosmap/data/gene_exp/DLPFC/geneTpmResidualsAgeGenderAdj_rename.txt",
                     "/home/hs3163/Project/Rosmap/data/gene_exp/PCC/geneTpmResidualsAgeGenderAdj_rename.txt"],
                "region_list" : ["~/GIT/ADSPFG-xQTL/MWE/mwe_region",
                "~/GIT/ADSPFG-xQTL/MWE/mwe_region",
                "~/GIT/ADSPFG-xQTL/MWE/mwe_region"] ,
                "covariate_file" : ["~/GIT/ADSPFG-xQTL/MWE/mwe_cov","~/GIT/ADSPFG-xQTL/MWE/mwe_cov","~/GIT/ADSPFG-xQTL/MWE/mwe_cov"],
                "factor_analysis_opt" : ["APEX","APEX","APEX"],
                "LD_options": ["In-sample","In-sample","In-sample"],
                "QTL_tool_option" : ["APEX","TensorQTL","APEX"],
                "QTL_analysis_option" : ["cis","cis","cis"],
                "cis_windows" : [500000,500000,5000000],
                "Metal" : ["T","T","F"]})

In [8]:
Recipe_temp

,Theme,genotype_list,molecular_pheno,region_list,covariate_file,factor_analysis_opt,LD_options,QTL_tool_option,QTL_analysis_option,cis_windows,Metal
0,AC,/home/hs3163/GIT/ADSPFG-xQTL/MWE/mwe_genotype_...,/home/hs3163/Project/Rosmap/data/gene_exp/AC/g...,~/GIT/ADSPFG-xQTL/MWE/mwe_region,~/GIT/ADSPFG-xQTL/MWE/mwe_cov,APEX,In-sample,APEX,cis,500000,T
1,DLPFC,/home/hs3163/GIT/ADSPFG-xQTL/MWE/mwe_genotype_...,/home/hs3163/Project/Rosmap/data/gene_exp/DLPF...,~/GIT/ADSPFG-xQTL/MWE/mwe_region,~/GIT/ADSPFG-xQTL/MWE/mwe_cov,APEX,In-sample,TensorQTL,cis,500000,T
2,PCC,/home/hs3163/GIT/ADSPFG-xQTL/MWE/mwe_genotype_...,/home/hs3163/Project/Rosmap/data/gene_exp/PCC/...,~/GIT/ADSPFG-xQTL/MWE/mwe_region,~/GIT/ADSPFG-xQTL/MWE/mwe_cov,APEX,In-sample,APEX,cis,5000000,F


In [ ]:
library("dplyr")
library("readr")
library("purrr")
## Define a wrapper, modified from the gwasvcf packages, to create the vcf of needed.

    create_vcf = function (chrom, pos, nea, ea, snp = NULL, ea_af = NULL, effect = NULL,
    se = NULL, pval = NULL, n = NULL, ncase = NULL, name = NULL)
{
    stopifnot(length(chrom) == length(pos))
    if (is.null(snp)) {
        snp <- paste0(chrom, ":", pos)
    }
    snp <- paste0(chrom, ":", pos)
    nsnp <- length(chrom)
    gen <- list()
    ## Setupt data content for each sample column
    if (!is.null(ea_af))
        gen[["AF"]] <- matrix(ea_af, nsnp)
    if (!is.null(effect))
        gen[["ES"]] <- matrix(effect, nsnp)
    if (!is.null(se))
        gen[["SE"]] <- matrix(se, nsnp)
    if (!is.null(pval))
        gen[["LP"]] <- matrix(-log10(pval), nsnp)
    if (!is.null(n))
        gen[["SS"]] <- matrix(n, nsnp)
    if (!is.null(ncase))
        gen[["NC"]] <- matrix(ncase, nsnp)
    gen <- S4Vectors::SimpleList(gen)

  ## Setup snps info for the fix columns
    gr <- GenomicRanges::GRanges(chrom, IRanges::IRanges(start = pos,
        end = pos + pmax(nchar(nea), nchar(ea)) - 1, names = snp))
  ## Setup meta informations
     coldata <- S4Vectors::DataFrame(Studies = name, row.names = name)
    hdr <- VariantAnnotation::VCFHeader(header = IRanges::DataFrameList(fileformat = S4Vectors::DataFrame(Value = "VCFv4.2",
        row.names = "fileformat")), sample = name)
    VariantAnnotation::geno(hdr) <- S4Vectors::DataFrame(Number = c("A",
        "A", "A", "A", "A", "A"), Type = c("Float", "Float",
        "Float", "Float", "Float", "Float"), Description = c("Effect size estimate relative to the alternative allele",
        "Standard error of effect size estimate", "-log10 p-value for effect estimate",
        "Alternate allele frequency in the association study",
        "Sample size used to estimate genetic effect", "Number of cases used to estimate genetic effect"),
        row.names = c("ES", "SE", "LP", "AF", "SS", "NC"))
  ## Save only the meta information in the sample columns
    VariantAnnotation::geno(hdr) <- subset(VariantAnnotation::geno(hdr),
        rownames(VariantAnnotation::geno(hdr)) %in% names(gen))
  ## Save VCF values
    vcf <- VariantAnnotation::VCF(rowRanges = gr, colData = coldata,
        exptData = list(header = hdr), geno = gen)
    VariantAnnotation::alt(vcf) <- Biostrings::DNAStringSetList(as.list(ea))
    VariantAnnotation::ref(vcf) <- Biostrings::DNAStringSet(nea)
  ## Write fixed values
    VariantAnnotation::fixed(vcf)$FILTER <- "PASS"
        return(sort(vcf))
    }

    input = readRDS('/mnt/mfs/statgen/xqtl_workflow_testing/testing_6/Meta_analysis/sumstat/mash_rds/ENSG00000141959.posterior.rds')
    input_effect = input$PosteriorMean
    if(is.null(input$PosteriorSD)){
       input$PosteriorSD = matrix(1,nrow = nrow(input_effect),ncol = ncol(input_effect) )
      }
    input_se = input$PosteriorSD
    df = tibble(snps = input$snps)
    df = df%>%mutate(    chr = map_dbl(snps,~read.table(text = .x,sep = ":",as.is = T)$V1),
                 pos_alt_ref = map_chr(snps,~read.table(text = .x,sep = ":",as.is = TRUE)$V2),
                 pos = map_dbl(pos_alt_ref,~read.table(text = .x,sep = "_",as.is = TRUE)$V1),
                 alt = map_chr(pos_alt_ref,~read.table(text = .x,sep = "_",as.is = TRUE, colClass = "character")$V2),
                 ref = map_chr(pos_alt_ref,~read.table(text = .x,sep = "_",as.is = TRUE, colClass = "character")$V3))


    vcf = create_vcf(
       chrom = df$chr,
        pos = df$pos,
        ea = df$alt,
        nea = df$ref,
        effect = input_effect ,
        se =  input_se,
        name = colnames(input_effect))
      
  VariantAnnotation::writeVcf(vcf,'/mnt/mfs/statgen/xqtl_workflow_testing/testing_6/Meta_analysis/mash_vcf/mash_vcf/ENSG00000141959.posterior.vcf',index = TRUE)